In [19]:
import cv2
import numpy as np
from tensorflow import keras

path = './data/'
train_data = keras.preprocessing.image_dataset_from_directory(
    path,
    batch_size=32,
    image_size=(48, 48),
    validation_split=0.2,
    subset='training',
    seed=123
)

val_data = keras.preprocessing.image_dataset_from_directory(
    path,
    batch_size=32,
    image_size=(48, 48),
    validation_split=0.2,
    subset='validation',
    seed=123
)

model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(48, 48, 3)),
    keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(2, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_data, validation_data=val_data, epochs=10)

model.save('model.h5')

model = keras.models.load_model('model.h5')

def get_face(image):
    face_cascade = cv2.CascadeClassifier('../DATA/haarcascades/haarcascade_frontalface_default.xml')

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

    if len(faces) == 0:
        return None, None

    (x, y, w, h) = faces[0]

    return cv2.resize(image[y:y+w, x:x+h], (48, 48)), faces[0]

image = cv2.imread('s20/user.1.1 (10).jpg')

face, coords = get_face(image)

if face is not None:
    face_array = np.array(face, dtype=np.float32) / 255.0
face_array = np.expand_dims(face_array, axis=0)

prediction = model.predict(face_array)[0]


if prediction[0] > prediction[1]:
    label = '1'
else:
    label = 'None'

cv2.rectangle(image, (coords[0], coords[1]), (coords[0]+coords[2], coords[1]+coords[3]), (0, 255, 0), 2)
cv2.putText(image, label, (coords[0], coords[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

face_array = np.expand_dims(face_array, axis=0)

prediction = model.predict(face_array)[0]

if prediction[0] > prediction[1]:
    label = '1'
else:
    label = 'None'

cv2.rectangle(image, (coords[0], coords[1]), (coords[0]+coords[2], coords[1]+coords[3]), (0, 255, 0), 2)
cv2.putText(image, label, (coords[0], coords[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

cv2.imshow('image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()



Found 10 files belonging to 1 classes.
Using 8 files for training.
Found 10 files belonging to 1 classes.
Using 2 files for validation.
Epoch 1/10
1/1 [==============================] - 3s 3s/step - loss: 0.5630 - accuracy: 1.0000 - val_loss: 0.1520 - val_accuracy: 1.0000
Epoch 2/10
1/1 [==============================] - 0s 152ms/step - loss: 0.1512 - accuracy: 1.0000 - val_loss: 0.0131 - val_accuracy: 1.0000
Epoch 3/10
1/1 [==============================] - 0s 140ms/step - loss: 0.0115 - accuracy: 1.0000 - val_loss: 4.1850e-04 - val_accuracy: 1.0000
Epoch 4/10
1/1 [==============================] - 0s 140ms/step - loss: 3.6285e-04 - accuracy: 1.0000 - val_loss: 7.3313e-06 - val_accuracy: 1.0000
Epoch 5/10
1/1 [==============================] - 0s 156ms/step - loss: 6.2734e-06 - accuracy: 1.0000 - val_loss: 5.9605e-08 - val_accuracy: 1.0000
Epoch 6/10
1/1 [==============================] - 0s 147ms/step - loss: 1.4901e-08 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000

InvalidArgumentError: Graph execution error:

Detected at node 'sequential_2/conv2d_6/Relu' defined at (most recent call last):
    File "C:\Users\Nitro5\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\Nitro5\anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\Nitro5\anaconda3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\Nitro5\anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Users\Nitro5\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "C:\Users\Nitro5\anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\Nitro5\anaconda3\lib\asyncio\base_events.py", line 601, in run_forever
      self._ru

In [32]:















import cv2
import os
import numpy as np
import tensorflow as tf

face_cascade = cv2.CascadeClassifier('../DATA/haarcascades/haarcascade_frontalface_default.xml')





data_path = 'data'
categories = os.listdir(data_path)
labels = [i for i in range(len(categories))]

label_dict = dict(zip(categories, labels))

# Создание тренировочного и тестового наборов данных
data = []
target = []

for category in categories:
    folder_path = os.path.join(data_path, category)
    img_names = os.listdir(folder_path)
        
    for img_name in img_names:
        img_path = os.path.join(folder_path, img_name)
        img = cv2.imread(img_path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        
        for (x, y, w, h) in faces:
            face_img = gray[y:y+w, x:x+w]
            resized = cv2.resize(face_img, (100, 100))
            data.append(resized)
            target.append(label_dict[category])

# Преобразование данных в массивы NumPy
data = np.array(data)/255.0
data = np.reshape(data, (data.shape[0], 100, 100, 1))
target = np.array(target)

# Разделение набора данных на тренировочный и тестовый
from sklearn.model_selection import train_test_split
train_data, test_data, train_target, test_target = train_test_split(data, target, test_size=0.2)

# Создание модели
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(200, (3, 3), activation='relu', input_shape=(100, 100, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(100, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(len(categories), activation='softmax')
])

# Компиляция модели
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Обучение модели
model.fit(train_data, train_target, epochs=10, validation_data=(test_data, test_target))

# Оценка точности
test_loss, test_acc = model.evaluate(test_data, test_target)
print('Test accuracy:', test_acc)

# Сохранение модели
model.save('my_model.h5')




Epoch 1/10
1/1 [==============================] - 4s 4s/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 1s 640ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 3/10
1/1 [==============================] - 1s 624ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 4/10
1/1 [==============================] - 1s 694ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 5/10
1/1 [==============================] - 1s 664ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 6/10
1/1 [==============================] - 1s 720ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 7/10
1/1 [==============================] - 1s 744ms/step - l

In [33]:
# Загрузка модели
new_model = tf.keras.models.load_model('my_model.h5')

# Классификация новых изображений
def classify(image_path):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in faces:
        face_img = gray[y:y+w, x:x+w]
        resized = cv2.resize(face_img, (100, 100))
        normalized = resized/255.0
        reshaped = np.reshape(normalized, (1, 100, 100, 1))
        result = new_model.predict(reshaped)
    
        label = np.argmax(result, axis=1)[0]
        confidence = np.max(result, axis=1)[0]
    
        return categories[label], confidence
# Пример использования
image_path = 'dogs.jpeg'
label, confidence = classify(image_path)
print('This person is', label, 'with', confidence*100, '% confidence')

TypeError: cannot unpack non-iterable NoneType object

In [34]:
# import all libraries
import cv2
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Model, load_model
from keras.layers import *
import uuid
import tensorflow as tf
from keras.metrics import Precision, Recall
import keras

In [35]:
# define hyperparameters
BATCH_SIZE = 16
EPOCHS = 15
POSITIVE_PATH = os.path.join("data", "positive")
NEGATIVE_PATH = os.path.join("data", "negative")
ANCHOR_PATH = os.path.join("data", "anchor")

In [37]:
# get images 
anchor_images = tf.data.Dataset.list_files('s20' + "\*jpg").take(3000)
positive_images = tf.data.Dataset.list_files(POSITIVE_PATH + "\*jpg").take(3000)
negative_images = tf.data.Dataset.list_files(NEGATIVE_PATH + "\*jpg").take(3000)

InvalidArgumentError: Expected 'tf.Tensor(False, shape=(), dtype=bool)' to be true. Summarized data: b'No files matched pattern: data\\positive\\*jpg'